### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [75]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [76]:
X_test.head(3)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,63,F,ATA,140,195,0,Normal,179,N,0.0,Up,0
1,53,M,NAP,145,518,0,Normal,130,N,0.0,Flat,1
2,65,M,ASY,160,0,1,ST,122,N,1.2,Flat,1


In [77]:
X_test.shape

(303, 12)

In [78]:
import urllib.request
import json      

def get_prediction(Age, Sex, ChestPainType, RestingBP, Cholesterol, FastingBS, RestingECG, MaxHR, ExerciseAngina, Oldpeak, ST_Slope, HeartDisease):
    body = {'Age': Age, 
            'Sex': Sex,
            'ChestPainType': ChestPainType,
            'RestingBP': RestingBP,
            'Cholesterol': Cholesterol,
            'FastingBS': FastingBS,
            'RestingECG': RestingECG,
            'MaxHR': MaxHR,
            'ExerciseAngina': ExerciseAngina,
            'Oldpeak': Oldpeak,
            'ST_Slope': ST_Slope,
            'HeartDisease': HeartDisease,
           } 

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [79]:
X_test[:1]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,63,F,ATA,140,195,0,Normal,179,N,0.0,Up,0


In [80]:
get_prediction(63, 'F', 'ATA', 140, 195, 0, 'Normal', 179, 'N', 0.0, 'Up', 0)

0.023691140412949542

In [81]:
X_test.iloc[:100]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,63,F,ATA,140,195,0,Normal,179,N,0.0,Up,0
1,53,M,NAP,145,518,0,Normal,130,N,0.0,Flat,1
2,65,M,ASY,160,0,1,ST,122,N,1.2,Flat,1
3,56,M,ASY,130,0,0,LVH,122,Y,1.0,Flat,1
4,54,M,ATA,108,309,0,Normal,156,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,55,M,ATA,140,196,0,Normal,150,N,0.0,Up,0
96,43,M,ASY,115,0,0,Normal,145,Y,2.0,Flat,1
97,74,M,NAP,138,0,0,Normal,116,N,0.2,Up,0
98,40,M,ASY,110,167,0,LVH,114,Y,2.0,Flat,1


In [82]:
%%time
predictions = X_test.iloc[:100].apply(lambda x: get_prediction(*(x[i] for i in range(len(x)))), 1)

CPU times: user 124 ms, sys: 19.5 ms, total: 144 ms
Wall time: 1.09 s


In [83]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:100])

0.921592775041051